<a href="https://colab.research.google.com/github/kumar-selvakumaran/pytorch_training/blob/main/NN_toys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Notes:</h1>
<h3>
1.   pytorch forum says to alter parameters manually only with torch.no_grad():, but tried doing it without. : <a href="https://discuss.pytorch.org/t/how-to-manually-set-the-weights-in-a-two-layer-linear-model/45902">link</a><br><br> dosent work : <br>error :

```
RuntimeError                              Traceback (most recent call last)

<ipython-input-160-fbab85b2caea> in <cell line: 1>()
----> 1 modelpart.get_parameter("layer.weight")[0][0][0,1] = 0
      2
      3 # print_pars(modelpart)

RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.
```
works only with torch.no_grad():<br>
maybe related to doing because its not recommended to do inplace opertaions as read in the documentation<br>
<hr>
2.    when using multiple loss functions, to back propogate on each of them, you can just add each of the component losses to make a final loss, and do a final_loss.backward(); optimizer.step(). and this updates all the parameters according to how they contribute to each of these losses. i.e. as intended.
<hr>
4.   Parameters init done in nanogpt repo
<a href="https://github.com/karpathy/nanoGPT/blob/master/model.py">link</a><br>
will help in
convergence.

<hr>
5.   **DOUBT** : **WHY IS THIS HAPENING** :gradient descent step, not multiplying the loss with the gradient, it is doing w_new = w + (alpha * del_J_wrt_w), instead of w_new = w + (alpha * del_J_wrt_w * **J** )

**CONCLUSION : UPDATE WAS NEVER  w_new = w + (alpha * del_J_wrt_w * J ). IT WAS ALWAYS SUPPOSED TO BE w_new = w + (alpha * del_J_wrt_w). WHEN WE HAVE A POLYNOMIAL LOSS FUNCTION, LIKE MSE, OR SSE, THE DERIVATIVE GIVES US A COEFFICIENT OF J. IN THESE CASES, THE MAGNITUTED OF J IS BACK-PROPAGATED**.
<hr>

In [ ]:
#helper functions KEEP
#printing gradients and parameters of the model
def print_pars(model):
  named_pars = tuple(model.named_parameters())
  for ind, i in enumerate(named_pars):
    print(f"{i[0]} parameters : {i[1][0]} | gradients : {i[1].grad}")


def init_pars(model):
  with torch.no_grad():
    for name, parameter in model.named_parameters():
      parameter = model.get_parameter(name)
      if "bias" in name:
        parameter *= 0
      else:
        parameter = parameter ** 0
      model.get_parameter(name)[0] = parameter


<hr>

<h3> Experiment 1: understanding the compuation graph of convolutional operator, and an optimization step </h3>

<h3>- link to image with computation graph of the convolutional operator :

[link](https://drive.google.com/file/d/12YDishmlx_VFfI00QXFR7CuZ4VCSpe0z/view?usp=drive_link)


In [ ]:
import torch
from torch import nn
from torch import optim

#helper functions KEEP

#printing gradients and parameters of the model
def print_pars(model):
  named_pars = tuple(model.named_parameters())
  for ind, i in enumerate(named_pars):
    print(f"{i[0]} parameters : {i[1][0]} | gradients : {i[1].grad}")

#initializing the weights to 1 and bias to 0
def init_pars(model):
  with torch.no_grad():
    for name, parameter in model.named_parameters():
      parameter = model.get_parameter(name)
      if "bias" in name:
        parameter *= 0
      else:
        parameter = parameter ** 0
      model.get_parameter(name)[0] = parameter

#model used
model = nn.Conv1d(in_channels = 1,
                  out_channels = 1,
                  kernel_size = 2)

#initializing model weights to 1, and bias to 0
init_pars(model)

optimizer = optim.SGD(model.parameters(), lr = 1)

model.train()


inp = torch.ones([1,1,3])

#using the same input for all iterations
print(f"INPUT IS ALWAYS : {inp}\n")

for _ in range(10):
  gt = torch.ones([1,1,2]) * 17    #results are same , regardless of the loss value
  gt.requires_grad = True
  print(f"--------------------")
  print(f"old parameters :")
  print_pars(model)
  old_param = model.get_parameter("weight")[0].clone().detach()
  pred = model(inp)
  print(f"\npred : {pred},\n gt : {gt}\n")
  lossval = (gt.sum() - pred.sum()).abs()
  optimizer.zero_grad()
  lossval.backward()
  optimizer.step()
  new_param = model.get_parameter("weight")[0].clone().detach()
  print(f"(update check) old_param {old_param} - gradients {model.get_parameter('weight').grad} == new_params {new_param}\n")
  print(f"updated parameters:")
  print_pars(model)

INPUT IS ALWAYS : tensor([[[1., 1., 1.]]])

--------------------
old parameters :
weight parameters : tensor([[1., 1.]], grad_fn=<SelectBackward0>) | gradients : None
bias parameters : -0.0 | gradients : None

pred : tensor([[[2., 2.]]], grad_fn=<ConvolutionBackward0>),
 gt : tensor([[[17., 17.]]], requires_grad=True)

(update check) old_param tensor([[1., 1.]]) - gradients tensor([[[-2., -2.]]]) == new_params tensor([[3., 3.]])

updated parameters:
weight parameters : tensor([[3., 3.]], grad_fn=<SelectBackward0>) | gradients : tensor([[[-2., -2.]]])
bias parameters : 2.0 | gradients : tensor([-2.])
--------------------
old parameters :
weight parameters : tensor([[3., 3.]], grad_fn=<SelectBackward0>) | gradients : tensor([[[-2., -2.]]])
bias parameters : 2.0 | gradients : tensor([-2.])

pred : tensor([[[8., 8.]]], grad_fn=<ConvolutionBackward0>),
 gt : tensor([[[17., 17.]]], requires_grad=True)

(update check) old_param tensor([[3., 3.]]) - gradients tensor([[[-2., -2.]]]) == new_para

In [ ]:
model(inp)

<hr>

<h3> Experiment 2: multiple heads, and loss functions </h3>

<h3>- link to image with computation graph of multple loss functions :

[link](https://drive.google.com/file/d/1CAHbP0r1m_LHgJ8ASeCQb2Axug049RdF/view?usp=drive_link)



In [ ]:
import torch
from torch import nn


class convlayer(nn.Module):
  def __init__(self):
    super(convlayer, self).__init__()
    self.layer = nn.Conv2d(in_channels = 1,
                           out_channels = 1,
                           kernel_size = 2)
  def forward(self, x):
    return self.layer(x)


class toynn(nn.Module):
  def __init__(self):
    super().__init__()

    conv1 = convlayer()
    self.conv1 = conv1

    leaf1 = convlayer()
    self.leaf1 = leaf1

    leaf2 = convlayer()
    self.leaf2 = leaf2

  def forward(self, x):
    x = self.conv1(x)
    x1 = self.leaf1(x)
    x2 = self.leaf2(x)
    ypred = torch.cat((x1, x2), axis = 0)
    return ypred



x = torch.ones((1,1,3,3))
model = toynn()

with torch.no_grad():
  model.get_parameter("leaf1.layer.weight")[0][0][0,0] = 1
  model.get_parameter("leaf1.layer.bias")[0] = 0
  model.get_parameter("leaf1.layer.weight")[0][0][1,0] = 1
  model.get_parameter("leaf1.layer.weight")[0][0][0,1] = 1
  model.get_parameter("leaf1.layer.weight")[0][0][1,1] = 1

  model.get_parameter("leaf2.layer.weight")[0][0][0,0] = 1
  model.get_parameter("leaf2.layer.bias")[0] = 0
  model.get_parameter("leaf2.layer.weight")[0][0][1,0] = 1
  model.get_parameter("leaf2.layer.weight")[0][0][0,1] = 1
  model.get_parameter("leaf2.layer.weight")[0][0][1,1] = 1




  model.get_parameter("conv1.layer.weight")[0][0][0,0] = 1
  model.get_parameter("conv1.layer.bias")[0] = 0
  model.get_parameter("conv1.layer.weight")[0][0][1,0] = 1
  model.get_parameter("conv1.layer.weight")[0][0][0,1] = 1
  model.get_parameter("conv1.layer.weight")[0][0][1,1] = 1

y = model(x)
print(y)

def ownloss(gt, pred):
  return gt - pred

def ownloss2(gt, pred):
  return -(gt @ pred)

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1)

model.train()

gt = torch.tensor([[[17.0]], [[3.0]]])
gt.requires_grad = True

lossval1 = ownloss(gt[0],y[0])
lossval2 = ownloss2(gt[1],y[1])
# lossval1.backward(retain_graph = True)
# lossval2.backward()
lv = lossval1+lossval2
lv.backward()
optimizer.step()
print_pars(model)




---



<h3>Experiment 3 : Feature pyramid networks toy</h1>

In [ ]:
import torch
from torch import nn
from torch import optim


class layer_block(nn.Module):
  def __init__(self):
    super(layer_block, self).__init__()
    self.block = nn.Conv2d(in_channels = 1,
                           out_channels = 1,
                           kernel_size = 2,
                           stride = 2)
    self.output = None

  def forward(self, x):
    output = self.block(x)
    self.output = output
    print(f"\noutput : \n\n{output}\n")
    return output

class toyfpn(nn.Module):
  def __init__(self):
    super(toyfpn, self).__init__()

    self.topdown1 = layer_block()
    self.topdown2 = layer_block()
    self.topdown3 = layer_block()

  def upsample(self, x, mode = 'bilinear'):
    x = upsampled = nn.functional.interpolate(input = x,
                                              scale_factor = (2,2),
                                              mode = mode
                                              )
    return x

  def forward(self, x):
    outputs = tuple()
    print(f"\n(TOP DOWN)\n")
    print(f"\ninput : \n\n{x}\n")
    x = self.topdown1(x)
    x = self.topdown2(x)
    x = self.topdown3(x)

    outputs += tuple([x])

    print(f"\n(BOTTOM UP) \n")
    x = self.upsample(x)

    print(f"\n upsampled data : \n\n{x}, \n\n cross data : \n\n{self.topdown2.output}\n")

    x += self.topdown2.output

    outputs += tuple([x])

    x = self.upsample(x)

    print(f"\n upsampled data : \n\n{x}, \n\n cross data : \n\n{self.topdown1.output}")

    x += self.topdown1.output

    outputs += tuple([x])

    print(f"\n(OUTPUTS) : \n\n {outputs}\n")
    return outputs


model = toyfpn()

init_pars(model)

x = torch.ones((1,1,8,8))

pred = model(x)

In [ ]:
print_pars(model)